# 🧠 Extracción de Comentarios de YouTube sobre la Guerra en Ucrania (2024)


Este notebook busca videos relevantes de canales clasificados (pro-Ucrania, pro-Rusia, noticieros) y extrae **todos los comentarios top-level**.

Incluye:
- Clasificación ideológica de canales.
- Extracción de metadatos completos del video.
- Guardado incremental y tolerancia a errores.
- Control de videos ya procesados.

---

🔐 **Requisitos**: una clave de API de YouTube con 10.000 unidades diarias de cuota (gratuita por defecto).


In [1]:
import os
import pandas as pd
import time
import json
import re
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from dotenv import load_dotenv

ModuleNotFoundError: No module named 'dotenv'

In [6]:
load_dotenv(override=True)  # Parámetro clave para sobrescribir si ya estaba cargada la api key

# La API KEY está expuesta en el archivo .env solo con fines académicos y será eliminada 14 días después de la entrega
api_key = os.getenv('YOUTUBE_API_KEY')

In [3]:
# Nombres de canales y su clasificación
nombres_canales = {
    "Memorias de Pez": "pro-ucraniano",
    "VisualPolitik": "pro-ucraniano",
    "El País": "noticiero",
    "RTVE Noticias": "noticiero",
    "La Vanguardia": "noticiero",
    "Negocios TV": "pro-ruso",
    "Miguel Ruiz Calvo": "pro-ruso",
    "Intereconomía": "pro-ruso",
    "El Mundo": "noticiero",
    "laSexta Noticias": "noticiero"
    # "teleSUR": "pro-ruso"  <-- eliminado por visibilidad limitada
}


In [7]:
youtube = build('youtube', 'v3', developerKey=api_key)

In [22]:
def construir_diccionario_canales(youtube, nombres_clasificados):
    canales_interes = {}

    for nombre_objetivo, clasificacion in nombres_clasificados.items():
        try:
            respuesta = youtube.search().list(
                part="snippet",
                q=nombre_objetivo,
                type="channel",
                maxResults=1
            ).execute()

            item = respuesta["items"][0]
            canal_id = item["snippet"]["channelId"]
            canal_nombre = item["snippet"]["title"]

            canales_interes[canal_id] = {
                "nombre": canal_nombre,
                "clasificacion": clasificacion
            }

        except:
            continue  # Ignorar errores y pasar al siguiente

    return canales_interes



## Usa video list, aunque no me acuerdo de eso... Será que no es el original?

In [35]:
def buscar_videos_2024(youtube, canales_interes, 
                       published_after="2024-01-01T00:00:00Z",
                       published_before="2025-01-01T00:00:00Z"):
    video_list = []

    for canal_id, datos in canales_interes.items():
        next_page_token = None

        while True:
            request = youtube.search().list(
                part="id,snippet",
                channelId=canal_id,
                maxResults=50,
                order="date",
                type="video",
                publishedAfter=published_after,
                publishedBefore=published_before,
                pageToken=next_page_token
            )
            response = request.execute()

            for item in response.get("items", []):
                video_info = {
                    "video_id": item["id"]["videoId"],
                    "title": item["snippet"]["title"],
                    "publishedAt": item["snippet"]["publishedAt"],
                    "channel_id": canal_id,
                    "channel_title": datos["nombre"],
                    "clasificacion": datos["clasificacion"]
                }
                video_list.append(video_info)

            next_page_token = response.get("nextPageToken")
            if not next_page_token:
                break

    return video_list


## Lo que sigue parece ser el oficial

In [ ]:
def extraer_comentarios_en_memoria(video_list, api_key, save_every=10000):
    youtube = build('youtube', 'v3', developerKey=api_key)
    comentarios_totales = []
    total_extraidos = 0

    # Ruta fija para archivo temporal
    tmp_path = "../data/raw/0_comments_raw.csv"


    for video in video_list:
        video_id = video['video_id']
        try:
            video_response = youtube.videos().list(
                part='snippet,statistics,contentDetails',
                id=video_id
            ).execute()

            item = video_response['items'][0]
            snippet = item['snippet']
            stats = item.get('statistics', {})
            content = item.get('contentDetails', {})

            video_data = {
                'video_title': snippet['title'],
                'channel_title': snippet['channelTitle'],
                'video_published_at': snippet['publishedAt'],
                'video_views': stats.get('viewCount', 0),
                'video_likes': stats.get('likeCount', 0),
                'video_duration': content.get('duration', ''),
                'video_tags': snippet.get('tags', []),
                'video_category_id': snippet.get('categoryId', ''),
                'relacion_evento': video.get('relacion_evento', ''),
                'evento': video.get('evento', ''),
                'tipo_evento': video.get('tipo_evento', ''),
                'condiciones_cuenta': video.get('condiciones_cuenta', '')
            }

            # Extraer comentarios (solo top-level)
            next_page_token = None
            while True:
                comment_response = youtube.commentThreads().list(
                    part='snippet',
                    videoId=video_id,
                    maxResults=100,
                    pageToken=next_page_token
                ).execute()

                for item in comment_response.get('items', []):
                    top_comment = item['snippet']['topLevelComment']['snippet']
                    comentarios_totales.append({
                        'comment_id': item['snippet']['topLevelComment']['id'],
                        'comment': top_comment['textDisplay'],
                        'user_id': top_comment['authorChannelId']['value'],
                        'user_name': top_comment['authorDisplayName'],
                        'comment_time': top_comment['publishedAt'],
                        'comment_likes': top_comment['likeCount'],
                        'total_reply_count': item['snippet']['totalReplyCount'],
                        'is_top_level_comment': True,
                        **video_data
                    })

                next_page_token = comment_response.get('nextPageToken')
                if not next_page_token:
                    break

                time.sleep(0.5)

        except HttpError as e:
            print(f"❌ Error con video {video_id}: {e}")
            continue

        total_extraidos += 1

        # 💾 Guardado temporal cada X comentarios
        if len(comentarios_totales) >= save_every:
            print(f"💾 Guardando temporalmente {len(comentarios_totales)} comentarios...")
            df_tmp = pd.DataFrame(comentarios_totales)
            df_tmp.to_csv(tmp_path, mode='a', index=False, header=not os.path.exists(tmp_path))
            comentarios_totales = []

        time.sleep(1)

    # Guardar lo que quede
    if comentarios_totales:
        print(f"💾 Guardando restantes {len(comentarios_totales)} comentarios finales...")
        df_tmp = pd.DataFrame(comentarios_totales)
        df_tmp.to_csv(tmp_path, mode='a', index=False, header=not os.path.exists(tmp_path))

    print(f"✅ Extracción completa. Todo guardado en: {tmp_path}")

## Termina acá -------------

In [ ]:
def filtrar_videos_relevantes(video_list, keywords=None):
    if keywords is None:
        keywords = ['ucrania', 'rusia', 'guerra', 'putin', 'zelenski', 'donbás', 'crimea']

    df = pd.DataFrame(video_list)
    df["video_title_lower"] = df["video_title"].str.lower()
    df["video_title_sin_puntuacion"] = df["video_title_lower"].apply(lambda x: re.sub(r"[^\w\s]", "", x))

    # Aplicar filtro inclusivo por keywords
    filtro = df["video_title_sin_puntuacion"].apply(lambda t: any(k in t for k in keywords))
    df_filtrado = df[filtro].drop(columns=["video_title_lower", "video_title_sin_puntuacion"]).reset_index(drop=True)

    print(f"🎯 Videos relevantes encontrados: {len(df_filtrado)} / {len(df)}")
    return df_filtrado.to_dict(orient="records")

In [12]:
# 3) Función para obtener fecha de creación en batch
def fetch_account_creation(youtube, ids):
    creation = {}
    for i in range(0, len(ids), 50):
        batch = ids[i:i+50]
        resp = youtube.channels().list(
            part="snippet",
            id=",".join(batch)
        ).execute()
        for item in resp.get('items', []):
            creation[item['id']] = item['snippet']['publishedAt']
        time.sleep(0.5)
    return creation

In [23]:
canales_interes = construir_diccionario_canales(youtube, nombres_canales)


In [24]:
canales_interes

{'UCPH3Oz99Y_jrVBCQMjQZNSg': {'nombre': 'Memorias de Pez',
  'clasificacion': 'pro-ucraniano'},
 'UCJQQVLyM6wtPleV4wFBK06g': {'nombre': 'VisualPolitik',
  'clasificacion': 'pro-ucraniano'},
 'UCnsvJeZO4RigQ898WdDNoBw': {'nombre': 'EL PAÍS',
  'clasificacion': 'noticiero'},
 'UC7QZIf0dta-XPXsp9Hv4dTw': {'nombre': 'RTVE Noticias',
  'clasificacion': 'noticiero'},
 'UClLLRs_mFTsNT5U-DqTYAGg': {'nombre': 'La Vanguardia',
  'clasificacion': 'noticiero'},
 'UCwd8Byi93KbnsYmCcKLExvQ': {'nombre': 'Negocios TV',
  'clasificacion': 'pro-ruso'},
 'UCgms7r9SaeYhuIBaPGOjnhw': {'nombre': 'Miguel Ruiz Calvo',
  'clasificacion': 'pro-ruso'},
 'UCNKomgId0-uTA-vVLM9v1pw': {'nombre': 'Intereconomía',
  'clasificacion': 'pro-ruso'},
 'UCGXbLrVe8vnkiFv7q2vYv3w': {'nombre': 'El Mundo',
  'clasificacion': 'noticiero'},
 'UCCJs5mITIqxqJGeFjt9N1Mg': {'nombre': 'laSexta Noticias',
  'clasificacion': 'noticiero'}}

In [25]:
channel_ids = list(canales_interes.keys())

In [26]:
# 3) Consulta estadísticas en batch (hasta 50 IDs por llamada)
response = youtube.channels().list(
    part='statistics',
    id=','.join(channel_ids)
).execute()

In [27]:
# 4) Mapea cada ID a su subscriberCount
subs_count = {
    item['id']: int(item['statistics'].get('subscriberCount', 0))
    for item in response.get('items', [])
}

In [35]:
title_to_id = {info['nombre']: cid for cid, info in canales_interes.items()}


In [ ]:
# Paso 1: Buscar videos
video_list = buscar_videos_2024(youtube, canales_interes)


In [13]:
df_videos = pd.DataFrame(video_list)
df_videos.shape
df_videos.head()


,video_id,channel_id,channel_title,video_title,video_published_at,evento,tipo_evento,relacion_evento,condiciones_cuenta
0,Ljk6OkssuqM,UCPH3Oz99Y_jrVBCQMjQZNSg,Memorias de Pez,✅ REWIND 2024: el resumen GEOPOLÍTICO del año ...,2024-12-31T12:32:02Z,,,,pro-ucraniano
1,n1UE2XUZ8MM,UCPH3Oz99Y_jrVBCQMjQZNSg,Memorias de Pez,✅¿Cómo era la TERRIBLE VIDA de un SOLDADO de l...,2024-12-28T14:25:32Z,,,,pro-ucraniano
2,8JilB1JcHdo,UCPH3Oz99Y_jrVBCQMjQZNSg,Memorias de Pez,✅ RESUMEN de la SEMANA 148 de guerra entre UCR...,2024-12-27T15:54:06Z,,,,pro-ucraniano
3,e6KNYXZtN30,UCPH3Oz99Y_jrVBCQMjQZNSg,Memorias de Pez,✅El DICTADOR más EXCÉNTRICO de LIBIA | La hist...,2024-12-23T17:34:36Z,,,,pro-ucraniano
4,rz2tHgcVz1c,UCPH3Oz99Y_jrVBCQMjQZNSg,Memorias de Pez,✅Los GRANDES MÁRTIRES de la historia,2024-12-21T12:31:54Z,,,,pro-ucraniano


In [14]:
if 'channel_id' in df_videos.columns:
    print(df_videos['channel_id'].value_counts())

    # O con nombres si están disponibles
    if 'channel_title' in df_videos.columns:
        print(df_videos['channel_title'].value_counts())


channel_id
UCgms7r9SaeYhuIBaPGOjnhw    262
UCPH3Oz99Y_jrVBCQMjQZNSg    253
UCCJs5mITIqxqJGeFjt9N1Mg     44
UCcgqSM4YEo5vVQpqwN-MaNw      4
UC7QZIf0dta-XPXsp9Hv4dTw      1
Name: count, dtype: int64
channel_title
Miguel Ruiz Calvo    262
Memorias de Pez      253
laSexta Noticias      44
Rihanna                4
RTVE Noticias          1
Name: count, dtype: int64


In [ ]:
video_list_filtrados = filtrar_videos_relevantes(video_list)

In [29]:
with open("../data/raw/video_list_full.json", "w", encoding="utf-8") as f:
    json.dump(video_list, f, ensure_ascii=False, indent=2)

In [ ]:
df_comentarios = extraer_comentarios_en_memoria(
    video_list_filtrados,
    api_key
)

📡 Extrayendo comentarios:   0%|          | 0/402 [00:00<?, ?it/s]

💾 Guardando temporalmente 10062 comentarios...
💾 Guardando temporalmente 10166 comentarios...
💾 Guardando temporalmente 10284 comentarios...
💾 Guardando temporalmente 10074 comentarios...
💾 Guardando temporalmente 10642 comentarios...
💾 Guardando temporalmente 10119 comentarios...
💾 Guardando temporalmente 10197 comentarios...
💾 Guardando temporalmente 10679 comentarios...
💾 Guardando temporalmente 10325 comentarios...
💾 Guardando temporalmente 10188 comentarios...
💾 Guardando temporalmente 10377 comentarios...
💾 Guardando restantes 684 comentarios finales...
✅ Extracción completa. Todo guardado en: comentarios_tmp.csv


# Pasaje del raw a clean abajo --> 

In [9]:
df = pd.read_csv("../data/raw/0_comments_raw.csv")
df_deduplicated = df.drop_duplicates(subset=["comment_id"])
print(f"Comentarios únicos: {len(df_deduplicated)}")

Comentarios únicos: 113776


In [10]:
df.shape

(113797, 21)

In [14]:
# 2) Toma los user_id únicos
user_ids = df_deduplicated['user_id'].dropna().tolist()

In [43]:
# Crea una copia limpia
df_deduplicated = df_deduplicated.copy()

In [15]:
creation_dates = fetch_account_creation(youtube, user_ids)

In [44]:
# 5) Mapea de vuelta a tu DataFrame
df_deduplicated['account_created_at'] = df_deduplicated['user_id'].map(creation_dates)

In [45]:
# 4) Añadimos ambos campos al DataFrame principal
df_deduplicated.loc[:, 'channel_id'] = df_deduplicated['channel_title'].map(title_to_id)
df_deduplicated.loc[:, 'subscriber_count'] = df_deduplicated['channel_id'].map(subs_count)

In [46]:
df_deduplicated.shape

(113776, 24)

In [41]:
# 💾 Guardar dataset limpio en la carpeta del proyecto
output_path = "../data/processed/1_comments_youtube_clean.csv"
df_deduplicated.to_csv(output_path, index=False)

print(f"✅ Dataset limpio guardado en: {output_path}")
print(f"🔢 Comentarios únicos: {len(df_deduplicated)}")

✅ Dataset limpio guardado en: ../data/processed/1_comments_youtube_clean.csv
🔢 Comentarios únicos: 113776
